# Cleaning, Concatenating and Indexing

In [4]:
import pandas as pd
import csv

In [5]:
year = 2018

In [6]:
data_name = "TRI_{}_WI.csv"

In [7]:
data = pd.read_csv(data_name.format(year))
data.head()

,1. YEAR,2. TRIFD,3. FRS ID,4. FACILITY NAME,5. STREET ADDRESS,6. CITY,7. COUNTY,8. ST,9. ZIP,10. BIA,...,107. 8.3 - ENERGY RECOVER OF,108. 8.4 - RECYCLING ON SITE,109. 8.5 - RECYCLING OFF SIT,110. 8.6 - TREATMENT ON SITE,111. 8.7 - TREATMENT OFF SITE,112. PRODUCTION WSTE (8.1-8.7),113. 8.8 - ONE-TIME RELEASE,114. PROD_RATIO_OR_ ACTIVITY,115. 8.9 - PRODUCTION RATIO,Unnamed: 115
0,2018,54701WRRNV5200S,110002044831,WRR ENVIRONMENTAL SERVICES CO INC,5200 RYDER RD,EAU CLAIRE,EAU CLAIRE,WI,54701,NaN,...,0.0,47501.0,0.00,0.0,14250.00,61757.00,NaN,PRODUCTION,1.22,NaN
1,2018,53095LVLVL807PL,110000417120,SCHREIBER FOODS,807 PLEASANT VALLEY RD,WEST BEND,WASHINGTON,WI,53095,NaN,...,0.0,0.0,2.51,73653.0,0.00,73757.51,NaN,PRODUCTION,1.10,NaN
2,2018,54476JCSCM900AL,110012339415,MERRILL IRON & STEEL INC,900 ALDERSON ST,SCHOFIELD,MARATHON,WI,54476,NaN,...,0.0,0.0,50259.00,0.0,0.00,50676.48,NaN,PRODUCTION,0.73,NaN
3,2018,53074MLDDD71SUN,110041042895,MOLDED DIMENSIONS,701 SUNSET RD,PORT WASHINGTON,OZAUKEE,WI,53074,NaN,...,0.0,0.0,0.00,0.0,0.00,0.00,NaN,NaN,NaN,NaN
4,2018,53563XXNCH1012T,110000419903,EVONIK MATERIALS CORP,337 VINCENT ST,MILTON,ROCK,WI,53563,NaN,...,0.0,0.0,0.00,0.0,2501.91,2591.00,NaN,PRODUCTION,0.38,NaN


In [8]:
data["1. YEAR"]

0       2018
1       2018
2       2018
3       2018
4       2018
        ... 
2815    2018
2816    2018
2817    2018
2818    2018
2819    2018
Name: 1. YEAR, Length: 2820, dtype: int64

In [9]:
clean_data = data[['1. YEAR', "7. COUNTY", "12. LATITUDE", "13. LONGITUDE", "19. INDUSTRY SECTOR", "100. TOTAL RELEASES"]]

In [10]:
clean_data = clean_data.rename(columns={'1. YEAR':'Year', "7. COUNTY":"County", "12. LATITUDE": "Latitude", "13. LONGITUDE":"Longitude", "19. INDUSTRY SECTOR":"Industry Sector", "100. TOTAL RELEASES":"Total Release"})
clean_data

,Year,County,Latitude,Longitude,Industry Sector,Total Release
0,2018,EAU CLAIRE,44.756796,-91.456350,Hazardous Waste,6.000
1,2018,WASHINGTON,43.351570,-88.078200,Food,102.000
2,2018,MARATHON,44.920350,-89.592840,Fabricated Metals,417.480
3,2018,OZAUKEE,43.376100,-87.884540,Chemicals,0.000
4,2018,ROCK,42.781111,-88.968889,Chemicals,89.090
...,...,...,...,...,...,...
2815,2018,SHEBOYGAN,43.674060,-87.781470,Chemicals,0.000
2816,2018,WINNEBAGO,44.070899,-88.527748,Chemicals,0.000
2817,2018,WINNEBAGO,44.070899,-88.527748,Chemicals,3983.000
2818,2018,WAUKESHA,43.000020,-88.120340,Fabricated Metals,6.000


In [11]:
clean_data.to_csv("TRI_{}_Clean.csv".format(year), index = False)

In [12]:
data2014 = pd.read_csv("TRI_{}_Clean.csv".format(2014))
data2015 = pd.read_csv("TRI_{}_Clean.csv".format(2015))
data2016 = pd.read_csv("TRI_{}_Clean.csv".format(2016))
data2017 = pd.read_csv("TRI_{}_Clean.csv".format(2017))
data2018 = pd.read_csv("TRI_{}_Clean.csv".format(2018))
frames = [data2014, data2015, data2016, data2017, data2018]

In [13]:
result = pd.concat(frames)
result.to_csv("complete_clean_data.csv", index = False)

# SQL Queries

In [18]:
from sqlalchemy import create_engine

In [19]:
engine = create_engine('sqlite://', echo=False)

In [20]:
df = pd.read_csv("complete_clean_data.csv")
df.to_sql("full_data", engine, if_exists='append', index=False)

In [24]:
def qry(sql, cap=10):
    return pd.read_sql(sql, engine).head(cap)

In [39]:
qry("""
SELECT County, AVG(`Total Release`) as `average`
FROM full_data
GROUP BY County
ORDER BY `average`
""", 9999)

,County,average
0,BAYFIELD,0.011000
1,IOWA,16.528667
2,WAUSHARA,47.764762
3,BURNETT,316.142857
4,RUSK,507.223077
...,...,...
61,WOOD,47580.172063
62,WAUPACA,55979.089674
63,BUFFALO,63513.250241
64,CLARK,69286.193333


In [37]:
qry("""
SELECT County
FROM full_data
GROUP BY County

""", 1000)

,County
0,ADAMS
1,ASHLAND
2,BARRON
3,BAYFIELD
4,BROWN
...,...
61,WAUKESHA
62,WAUPACA
63,WAUSHARA
64,WINNEBAGO
